先运行notebook `data_preprocess_1.ipynb`。

In [1]:
import os
import json
import datetime

import numpy as np
import pandas as pd

from pathlib import Path
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [2]:
data_fp = Path("C:/Users/xh_z/SynologyDrive/Projects/FinTechCaseStudies/CaseStudy3Sim")
data_fp_unzipped = data_fp/'.unzipped'

# 处理EPS数据

解析EPS(Earnings Per Share)数据。原始文件是按天存储的HTML文档，只不过文件扩展名是txt。

每天的HTML文档记录了当天发布的季度EPS信息，发布该信息的公司，以及是否超出分析师预期等信息。可查看示例文件`20021101.html`。

我们需要将每个HTML文档里的EPS表格解析出来。

In [3]:
def parse_table(table):
    rows = table.find_all('tr')
    # whether we have entered a panel
    panels_d = {
        'Upside\nSurprises': False, 
        'Met\nExpectations': False, 
        'Downside\nSurprises': False}
    if len(rows) == 0:
        return None
    # a table should contain at most three panels
    # keep track of at which row we enter a panel
    # and then drop those rows
    dropped = []
    for i, row in enumerate(rows):
        for panel in panels_d:
            if row.text.startswith(panel):
                # a panel should only be entered once
                assert not panels_d[panel] 
                panels_d[panel] = True
                dropped.append(i) # panel title row
                dropped.append(i+1) # panel header row
    if all([not met for panel, met in panels_d.items()]):
        # no panel exists
        return None
    # extract table column names, should be 
    # Company, Symbol, Surprise (%), Reported EPS, Consensus EPS, Earnings Call, More\nInfo
    # we don't use the last column
    colnames = [td.text for td in rows[1].find_all('td')][:-1]
    # remove panel title rows and headers
    rows = [[td.text for td in row.find_all('td')[:-1]] for i, row in enumerate(rows) if i not in dropped]
    # drop empty rows
    rows = [row for row in rows if len(row)>0]
    # convert to dataframe
    return pd.DataFrame(rows, columns=colnames)

In [4]:
# parse_table(tables[4])

In [5]:
eps_fp = data_fp/'eps.csv'
if os.path.isfile(eps_fp):
    eps_df = pd.read_csv(eps_fp)
else:
    # parse html files
    table_l = []
    for file in tqdm(os.listdir(data_fp_unzipped/'EPS')):
        if not file.endswith('.txt'):
            continue
        # parse one html file
        date = file.split('.')[0]
        with open(data_fp_unzipped/'EPS'/file, 'r') as f:
            raw_str = f.read()
        raw_str = raw_str.replace('<th', '<td')
        raw_str = raw_str.replace('</th>', '</td>')
        soup = BeautifulSoup(raw_str)
        for table in soup.find_all('table'):
            # parse one table in the current html file
            table = parse_table(table)
            if table is not None:
                # the parsed table is the target one
                # so we have got a dataframe
                # now add date information
                table['date'] = date
                table_l.append(table)
                break
    # concatenate all parsed tables as a big dataframe
    eps_df = pd.concat(table_l)
    eps_df.to_csv(data_fp/'eps.csv', index=False)
eps_df.head(1)

,Company,Symbol,Surprise (%),Reported EPS,Consensus EPS,Earnings Call,date
0,Apple Computer Inc,AAPL,6.67,0.16,0.15,NaN,20040114


In [6]:
eps_df.date = eps_df.date.astype(str)
eps_df.rename({'Surprise (%)': 'gap%'}, axis=1, inplace=True)
eps_df['gap'] = eps_df['Reported EPS'].astype(float) - eps_df['Consensus EPS'].astype(float)
eps_df.drop(['Consensus EPS', 'Reported EPS', 'Earnings Call'], axis=1, inplace=True)

In [7]:
eps_df.head(1)

,Company,Symbol,gap%,date,gap
0,Apple Computer Inc,AAPL,6.67,20040114,0.01


In [8]:
eps_df.shape

(23, 5)

# 定义公司集

读取标普500的公司列表。这个列表定期更新，我们使用的是2012年9月28日的版本。

In [9]:
snp500_df = pd.read_table(data_fp_unzipped/'snp_list'/'snp500_20120928.txt', header=None, names=['Company', 'Symbol', 'Industry'])

In [10]:
snp500_df.head(1)

,Company,Symbol,Industry
0,Apple Inc.,AAPL,Information Technology


In [11]:
snp500_df.shape

(500, 3)

只考虑标普500公司中同时有EPS以及8K文本的观测日。

In [12]:
# only consider S&P500 companies
eps_snp500_df = snp500_df[['Symbol', 'Industry']].merge(eps_df, on='Symbol', how='inner')
eps_snp500_df.head(1)

,Symbol,Industry,Company,gap%,date,gap
0,AAPL,Information Technology,Apple Computer Inc,6.67,20040114,0.01


In [13]:
eps_snp500_df.shape

(4, 6)

In [14]:
grped = eps_snp500_df.groupby('Symbol')
# load processed 8K reports
with open(data_fp/'.processed'/'AAPL.json', 'r') as f:
    firm_8K_data = json.load(f)

grp_pd_l = []
for symbol, grp_pd in tqdm(grped):
    print(f"processing {symbol}")
    # work with one S&P500 company
    json_fp = data_fp/'.processed'/(symbol+'.json')
    if not os.path.isfile(json_fp):
        continue
    # only deal with companies of which processed 8K reports are available
    with open(json_fp, 'r') as f:
        doc_d_l = json.load(f)
    date_l_8K = [doc_d['time'][:8] for doc_d in doc_d_l]
    date_l_eps = set(grp_pd['date'])
    # only consider 8K reports (days) of which eps data are available
    date_l = date_l_eps.intersection(date_l_8K)
    if len(date_l) == 0:
        continue
    doc_d_d = {date: doc_d for doc_d, date in zip(doc_d_l, date_l_8K) if date in date_l}
    # join eps data and 8K text data
    grp_pd = grp_pd[grp_pd['date'].isin(date_l)].copy()
    nrow = grp_pd.shape[0]
    grp_pd['events'] = [[] for _ in range(nrow)]
    for row_i, row in grp_pd.iterrows():
        doc_d = doc_d_d[row['date']]
        grp_pd.loc[row_i, 'time'] = doc_d['time']
        grp_pd.at[row_i, 'events'] = doc_d['events']
        if doc_d['seperated']:
            text = ' '.join([doc_d['body'][toc] for toc in doc_d['toc']])
        else:
            text = doc_d['body']['all']
        grp_pd.loc[row_i, 'text'] = text
        grp_pd.loc[row_i, 'doc_len'] = len(text.split())
    grp_pd_l.append(grp_pd)
# concatenate all companies' sub dataframe into one
merged_df = pd.concat(grp_pd_l, ignore_index=True)
# release memory
del grp_pd_l, firm_8K_data
merged_df['time'] = pd.to_datetime(merged_df.time)
merged_df.head(2)

  0%|          | 0/4 [00:00<?, ?it/s]

processing AAPL
processing INTC
processing TER
processing YHOO


,Symbol,Industry,Company,gap%,date,gap,events,time,text,doc_len
0,AAPL,Information Technology,Apple Computer Inc,6.67,20040114,0.01,[Financial statements and exhibits],2004-01-14 16:30:52,item financial statements and exhibits c exhib...,923.0


In [15]:
merged_df.shape

(1, 10)

# 定义标签

获取每天的标普500指数，数据集下载自[investing.com](https://www.investing.com/indices/us-spx-500-historical-data)，时间范围为1995-01-01至2016-01-01，保存到路径`data_fp`，文件名`S&P 500 Historical Data.csv`。

In [16]:
sp500index_df = pd.read_csv(data_fp/'S&P 500 Historical Data.csv', parse_dates=['Date'])
sp500index_df = sp500index_df.sort_values('Date', ascending=True)
sp500index_df['date'] = sp500index_df.Date.dt.strftime('%Y%m%d')
for col in ['Price', 'Open', 'High', 'Low']:
    sp500index_df[col] = sp500index_df[col].str.replace(',','').astype(float)
sp500index_df.rename({'Price': 'Close'}, axis=1, inplace=True)
sp500index_df['date_str'] = sp500index_df['Date'].dt.strftime('%Y%m%d')
sp500index_df = sp500index_df.set_index('date_str')
sp500index_df.head(1)

,Date,Close,Open,High,Low,Vol.,Change %,date
date_str,,,,,,,,
19950103,1995-01-03,459.1,459.2,459.3,457.2,-,-0.04%,19950103


加入股票市场数据并定义标签

定义开市时间，参考[stock market hours](https://stockanalysis.com/stock-market-hours/)。假设8K数据集里的时间都是美东时间（EST），我们将开市时间设定为每个交易日的 9:30 A.M. EST，闭市时间设定为 4:00 P.M. EST。

定义如下规则：

1. 如果8K报表发布于某个交易日，但是发布时间早于当天开市时间，则使用这个交易日的open price减去上个交易日的close price。
2. 如果8K报表发布于某个交易日，但是发布时间晚于当天闭市时间，则使用下个交易日的open price减去这个交易日的close price。
3. 如果8K报表发布于某个交易日，且发布时间为交易时间，则使用这个交易日的close price减去这个交易日的open price。
4. 如果8K报表发布日期不是任何交易日，则使用下个交易日的open price减去上个交易日的close price。

以上任意规则得到的价格变化均需减去同期标普500指数的变化。

In [17]:
grp_pd_l = []
for symbol, grp_pd in tqdm(merged_df.groupby('Symbol')):
    grp_pd = grp_pd.copy()
    # load stock data
    stock_fp = data_fp_unzipped/'price_history'/(symbol+'.csv')
    stock_df = pd.read_csv(stock_fp, parse_dates=['Date'])
    # sort by date in ascending order
    stock_df = stock_df.sort_values('Date', ascending=True)
    stock_df['date_str'] = stock_df['Date'].dt.strftime('%Y%m%d')
    stock_df = stock_df.set_index('date_str')
    # stock_df.head(1)
    # define label
    trade_days = stock_df.index.values
    for row_i, row in grp_pd.iterrows():
        pub_day = row['time'].strftime('%Y%m%d')
        if pub_day in trade_days:
            day_i = np.where(pub_day == trade_days)[0][0]
            trade_day = datetime.datetime.strptime(pub_day, '%Y%m%d')
            assert trade_day == stock_df.iloc[day_i]['Date']
            # market open time
            mo_time = trade_day + datetime.timedelta(hours=9, minutes=30) 
            # market close time
            mc_time = trade_day + datetime.timedelta(hours=16, minutes=0)
            if row['time'] <= mo_time:
                # rule 1, publish trade day open - previous trade day close
                p1 = stock_df.loc[trade_days[day_i], 'Open']
                p2 = stock_df.loc[trade_days[day_i-1], 'Close']
                s1 = sp500index_df.loc[trade_days[day_i], 'Open']
                s2 = sp500index_df.loc[trade_days[day_i], 'Close']
            elif row['time'] >= mc_time:
                # rule 2, next trade day open - publish trade day close
                p1 = stock_df.loc[trade_days[day_i+1], 'Open']
                p2 = stock_df.loc[trade_days[day_i], 'Close']
                s1 = sp500index_df.loc[trade_days[day_i+1], 'Open']
                s2 = sp500index_df.loc[trade_days[day_i], 'Close']
            else:
                # rule 3, publish trade day close - publish trade day open
                p1 = stock_df.loc[trade_days[day_i], 'Close']
                p2 = stock_df.loc[trade_days[day_i], 'Open']
                s1 = sp500index_df.loc[trade_days[day_i], 'Close']
                s2 = sp500index_df.loc[trade_days[day_i], 'Open']
        else:
            # rule 3, next trade day open - previous trade day close
            day_j = np.argmax(stock_df['Date'] > row['time'])
            p1 = stock_df.loc[trade_days[day_j], 'Open']
            p2 = stock_df.loc[trade_days[day_j-1], 'Close']
            s1 = sp500index_df.loc[trade_days[day_j], 'Open']
            s2 = sp500index_df.loc[trade_days[day_j-1], 'Close']
        grp_pd.loc[row_i, 'diff'] = (p1 - p2)/p2 - (s1-s2)/s2
        del p1, p2, s1, s2
    grp_pd_l.append(grp_pd)
    
final_df = pd.concat(grp_pd_l, ignore_index=True)
del grp_pd_l

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# define category label
cond1 = final_df['diff'] > 0.01
cond2 = final_df['diff'] < -0.01
final_df.loc[cond1, 'label'] = 2 # UP
final_df.loc[(~cond1)&(~cond2), 'label'] = 1 # STAY
final_df.loc[cond2, 'label'] = 0 # DOWN
assert final_df.label.isna().sum() == 0

In [19]:
final_df.head(1)

,Symbol,Industry,Company,gap%,date,gap,events,time,text,doc_len,diff,label
0,AAPL,Information Technology,Apple Computer Inc,6.67,20040114,0.01,[Financial statements and exhibits],2004-01-14 16:30:52,item financial statements and exhibits c exhib...,923.0,-0.053306,0.0


In [20]:
final_df.doc_len.describe(percentiles=[0.01])

count      1.0
mean     923.0
std        NaN
min      923.0
1%       923.0
50%      923.0
max      923.0
Name: doc_len, dtype: float64

In [21]:
min_n_word = 200
cond1 = final_df['doc_len'] == 0
cond2 = final_df['doc_len'] < min_n_word
print(cond1.sum(), 'firm-day obervations have zero document length')
print(cond2.sum(), f"firm-day obervations have less than {min_n_word} words")
save_df = final_df[~cond2]

0 firm-day obervations have zero document length
0 firm-day obervations have less than 200 words


In [22]:
save_df.shape

(1, 12)

In [23]:
cond1 = save_df.time.dt.year <= 2008
cond2 = save_df.time.dt.year >= 2011
print('train:', cond1.sum(), 'dev:', ((~cond1)&(~cond2)).sum(), 'test:', cond2.sum())

train: 1 dev: 0 test: 0


In [24]:
save_df.head(1)

,Symbol,Industry,Company,gap%,date,gap,events,time,text,doc_len,diff,label
0,AAPL,Information Technology,Apple Computer Inc,6.67,20040114,0.01,[Financial statements and exhibits],2004-01-14 16:30:52,item financial statements and exhibits c exhib...,923.0,-0.053306,0.0


In [25]:
save_df.to_csv('final_df.csv.gz', index=False)

In [26]:
!jupyter nbconvert --to html -TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' data_preprocess_2.ipynb

C:\Users\xh_z\anaconda3\lib\site-packages\traitlets\traitlets.py:2544: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'remove_cell' instead of "'remove_cell'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook data_preprocess_2.ipynb to html
[NbConvertApp] Writing 656176 bytes to data_preprocess_2.html
